In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm

# Create directory for saving generated images
output_dir = "generated_images3"
os.makedirs(output_dir, exist_ok=True)

# Custom Dataset class
class MRIDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []

        for root, _, files in os.walk(root_dir):
            for file in files:
                if file.endswith('.png'):
                    self.image_paths.append(os.path.join(root, file))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('L')  # Convert to grayscale
        
        if self.transform:
            image = self.transform(image)
        
        return image

# Generator Network
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 1, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x)

# Discriminator Network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x)

# Training function
def train_gan(dataloader, num_epochs, latent_dim, device):
    netG = Generator(latent_dim).to(device)
    netD = Discriminator().to(device)
    
    criterion = nn.BCELoss()
    optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))
    
    fixed_noise = torch.randn(64, latent_dim, 1, 1, device=device)

    for epoch in range(num_epochs):
        for i, data in enumerate(tqdm(dataloader)):
            batch_size = data.size(0)
            real_label = torch.ones(batch_size, device=device)
            fake_label = torch.zeros(batch_size, device=device)

            # Train Discriminator
            netD.zero_grad()
            real = data.to(device)
            output = netD(real).view(-1)
            errD_real = criterion(output, real_label)
            errD_real.backward()

            noise = torch.randn(batch_size, latent_dim, 1, 1, device=device)
            fake = netG(noise)
            output = netD(fake.detach()).view(-1)
            errD_fake = criterion(output, fake_label)
            errD_fake.backward()
            
            errD = errD_real + errD_fake
            optimizerD.step()

            # Train Generator
            netG.zero_grad()
            output = netD(fake).view(-1)
            errG = criterion(output, real_label)
            errG.backward()
            optimizerG.step()

        # Save generated images every epoch in the new folder
        if (epoch + 1) % 1 == 0:
            with torch.no_grad():
                fake = netG(fixed_noise)
                save_path = os.path.join(output_dir, f'fake_samples_epoch_{epoch+1}.png')
                torchvision.utils.save_image(fake.detach(), save_path, normalize=True)
            print(f"Saved image: {save_path}")

        print(f'Epoch [{epoch+1}/{num_epochs}] Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f}')
    
    return netG, netD

# Main execution
if __name__ == "__main__":
    # Hyperparameters
    latent_dim = 100
    image_size = 64
    batch_size = 64
    num_epochs = 300
    
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Data preprocessing
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    # Load dataset
    dataset = MRIDataset(root_dir='converted_images', transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    
    # Train the model
    generator, discriminator = train_gan(dataloader, num_epochs, latent_dim, device)
    
    # Save the models
    torch.save(generator.state_dict(), 'generator.pth')
    torch.save(discriminator.state_dict(), 'discriminator.pth')


100%|██████████| 62/62 [00:03<00:00, 17.57it/s]


Saved image: generated_images3/fake_samples_epoch_1.png
Epoch [1/300] Loss_D: 0.0026 Loss_G: 7.4099


100%|██████████| 62/62 [00:03<00:00, 17.94it/s]


Saved image: generated_images3/fake_samples_epoch_2.png
Epoch [2/300] Loss_D: 13.6470 Loss_G: 12.0244


100%|██████████| 62/62 [00:03<00:00, 17.82it/s]


...

Saved image: generated_images3/fake_samples_epoch_298.png
Epoch [298/300] Loss_D: 0.0410 Loss_G: 7.9813


100%|██████████| 62/62 [00:04<00:00, 15.43it/s]


Saved image: generated_images3/fake_samples_epoch_299.png
Epoch [299/300] Loss_D: 0.0313 Loss_G: 7.9297


100%|██████████| 62/62 [00:03<00:00, 15.71it/s]


Saved image: generated_images3/fake_samples_epoch_300.png
Epoch [300/300] Loss_D: 0.0566 Loss_G: 4.9512
